# Photometry using astropy photutils and AAVSO comparison stars

## 1 Import dependencies and setup matplotlib

In [ ]:
import math, glob
import pandas as pd
import numpy as np
import sep
from photutils.aperture import aperture_photometry
from photutils.aperture import CircularAperture
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import astropy.units as u
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


## 2  Define input file, star name and comparason star range¶

In [9]:
FITS_FILE = '/home/gtulloch/Projects/Photometry-Pipeline/data/BGO/RWAUR-ID12496-OC145880-GR5353-B.fit'
STAR_NAME = 'RW AUR'
BRIGHTEST_COMPARISON_STAR_MAG = 11.0
DIMMEST_COMPARISON_STAR_MAG = 13.0


## 3 Define a function to download comparison stars from AAVSO. 

This will return a tuple containing an array and the chart ID.

In [10]:
def get_comp_stars(ra,dec,filter_band='V',field_of_view=18.5):
    result = []
    vsp_template = 'https://www.aavso.org/apps/vsp/api/chart/?format=json&fov={}&maglimit=18.5&ra={}&dec={}'
    print(vsp_template.format(field_of_view, ra, dec))
    r = requests.get(vsp_template.format(field_of_view, ra, dec))
    chart_id = r.json()['chartid']
    print('Downloaded Comparison Star Chart ID {}'.format(chart_id))
    for star in r.json()['photometry']:
        comparison = {}
        comparison['auid'] = star['auid']
        comparison['ra'] = star['ra']
        comparison['dec'] = star['dec']
        for band in star['bands']:
            if band['band'] == filter_band:
                comparison['vmag'] = band['mag']
                comparison['error'] = band['error']
        result.append(comparison)
    return result, chart_id

## 4 Download comparison stars and search simbad for our target.

Use astroquery to locate the RA/DEC of our target star.

Here we also define `results` which will be a collection of data progressivly enriched as we go


In [11]:
astroquery_results = Simbad.query_object(STAR_NAME)
TARGET_RA = str(astroquery_results[0]['RA'])
TARGET_DEC = str(astroquery_results[0]['DEC']).replace('+','').replace('-','')
results, chart_id = get_comp_stars(TARGET_RA, TARGET_DEC)
print('{} comp stars found'.format(len(results)))
results.append({'auid': 'target', 'ra': TARGET_RA, 'dec': TARGET_DEC})

results

https://www.aavso.org/apps/vsp/api/chart/?format=json&fov=18.5&maglimit=18.5&ra=05 07 49.5662&dec=30 24 05.177
Downloaded Comparison Star Chart ID X34463DV
10 comp stars found


[{'auid': '000-BBH-921',
  'ra': '05:07:24.62',
  'dec': '30:20:28.1',
  'vmag': 9.617,
  'error': 0.04},
 {'auid': '000-BBH-941',
  'ra': '05:07:52.64',
  'dec': '30:30:33.0',
  'vmag': 11.057,
  'error': 0.019},
 {'auid': '000-BBH-947',
  'ra': '05:08:05.17',
  'dec': '30:18:40.6',
  'vmag': 11.437,
  'error': 0.011},
 {'auid': '000-BBH-938',
  'ra': '05:07:50.55',
  'dec': '30:19:02.5',
  'vmag': 12.046,
  'error': 0.012},
 {'auid': '000-BBH-935',
  'ra': '05:07:47.62',
  'dec': '30:26:58.7',
  'vmag': 12.707,
  'error': 0.006},
 {'auid': '000-BKL-839',
  'ra': '05:07:35.27',
  'dec': '30:24:47.0',
  'vmag': 12.901,
  'error': 0.017},
 {'auid': '000-BLV-683',
  'ra': '05:08:08.43',
  'dec': '30:22:08.8',
  'vmag': 13.09,
  'error': 0.006},
 {'auid': '000-BKL-841',
  'ra': '05:07:55.52',
  'dec': '30:16:36.3',
  'vmag': 13.464,
  'error': 0.017},
 {'auid': '000-BLV-684',
  'ra': '05:08:12.37',
  'dec': '30:24:35.6',
  'vmag': 13.472,
  'error': 0.0},
 {'auid': '000-BLV-685',
  'ra': 

## 5 Extract all sources from image

In [ ]:
# extract sources from image and add details to comp_stars
source_snr = 20
hdulist = fits.open(FITS_FILE)
data = hdulist[0].data.astype(np.float32)
header = hdulist[0].header
wcs = WCS(header)
bkg = sep.Background(data)
data_sub = data - bkg
objects = sep.extract(data_sub, source_snr * bkg.globalrms)
sources = objects
sources

id,xcentroid,ycentroid,sharpness,roundness1,roundness2,npix,sky,peak,flux,mag
int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64
1,947.861590909742,55.38494074002458,0.5355321525771656,0.29928891625487103,-0.06390957134245706,25,0.0,919.0,3.273571912221738,-1.2875547143272938
2,760.3162124695637,58.170167869150575,0.4210630809150676,0.09859457295402321,-0.432645420640777,25,0.0,431.0,1.1146339871775315,-0.11783070333781887
3,193.03342243823337,134.91093282246263,0.350784847353616,-0.10714575004217475,-0.30573102750045755,25,0.0,1306.0,5.1048940913174015,-1.7699668410653744
4,1366.3353645355774,145.25636804340616,0.5903375889151248,0.2872929039485848,0.08143188850620682,25,0.0,4369.0,17.703126355828616,-3.1201249228520127
5,789.0919390923975,147.0508607245247,0.5592908848406637,0.5639247150990722,-0.13897535957457352,25,0.0,431.0,1.0213748478071643,-0.022962896984002907
6,407.7099804140923,194.7475615205637,0.4939582017223762,0.2114153372814026,-0.006310581459245703,25,0.0,594.0,1.7223012945054808,-0.5902728200228635
7,1285.7764183646473,214.36240771038987,0.5578184837307454,0.34621808417576005,0.040549147868890426,25,0.0,720.0,2.4190341017444608,-0.9591049769144662
8,654.0820588145001,219.9891611246668,0.6077167529669182,0.5739519798975994,0.0828158506237739,25,0.0,3229.0,12.366654544843591,-2.7306305729978684
9,639.5520522876109,231.67053352009933,0.5615787477856459,0.8192746901072777,0.33959626808920046,25,0.0,1740.0,6.118081366017681,-1.9665381212169075


## 6 Find the sources that correspond to our target and comparison stars

Any source within 4 arc seconds of a comparison star is considered a match. Here the `results` will be converted to a pandas dataframe and will now contain the x,y coords of the target and comparison stars.

In [ ]:
print(wcs.naxis)
for star in results:
    star_coord = SkyCoord(star['ra'],star['dec'], unit=(u.hourangle, u.deg))
    xy = SkyCoord.to_pixel(star_coord, wcs=wcs, origin=1)
    x = xy[0].item(0)
    y = xy[1].item(0)
    for source in sources:
        # SEP returns source centroids in source['x'], source['y']
        if(source['x']-4 < x < source['x']+4) and source['y']-4 < y < source['y']+4:
            star['x'] = x
            star['y'] = y
            star['peak'] = source['peak']
results = pd.DataFrame(results)
results

2


,auid,ra,dec,vmag,error,x,y,peak
0,000-BBH-921,05:07:24.62,30:20:28.1,9.617,0.040,381.824848,500.040333,14169.0
1,000-BBH-941,05:07:52.64,30:30:33.0,11.057,0.019,769.838463,1147.365567,3537.0
2,000-BBH-947,05:08:05.17,30:18:40.6,11.437,0.011,943.707087,384.987222,2149.0
3,000-BBH-938,05:07:50.55,30:19:02.5,12.046,0.012,741.088988,408.329135,1469.0
4,000-BBH-935,05:07:47.62,30:26:58.7,12.707,0.006,700.432847,917.990542,932.0
5,000-BKL-839,05:07:35.27,30:24:47.0,12.901,0.017,529.471611,777.055729,962.0
6,000-BLV-683,05:08:08.43,30:22:08.8,13.090,0.006,988.692420,607.856477,613.0
7,000-BKL-841,05:07:55.52,30:16:36.3,13.464,0.017,810.023814,251.874422,566.0
8,000-BLV-684,05:08:12.37,30:24:35.6,13.472,0.000,1043.103696,765.025073,481.0
9,000-BLV-685,05:08:04.46,30:20:55.3,14.046,0.021,NaN,NaN,NaN


## 7 Plot the image annotated with the comparsion stars
This is just to check everything looks ok

In [52]:
# plot the image with overlay
results = results.query('x > 0 and y > 0') 

hdulist = fits.open(FITS_FILE)
data = hdulist[0].data.astype(float)
fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
positions = (results['x'], results['y'])    
position_apps = CircularAperture(positions, r=20.)    
# target_app = CircularAperture(target_xy, r=20.)    
plt.imshow(data, cmap='gray_r', origin='lower', vmin=0, vmax=2500)
position_apps.plot(color='red', lw=1.5, alpha=0.5)
# target_app.plot(color='blue', lw=1.5, alpha=0.5)
to_plot = results.query('peak < 50000 and vmag < 20 and vmag > 0') 
for to_annotate in results.iterrows():
    plt.annotate('{}'.format(to_annotate[1]['auid']),
        xy=(to_annotate[1]['x'], to_annotate[1]['y']), xycoords='data',
        xytext=(-150, 130), textcoords='offset points', size='16',
        arrowprops=dict(arrowstyle="->"))

(0      381.824848
1      769.838463
2      943.707087
3      741.088988
4      700.432847
5      529.471611
6      988.692420
7      810.023814
8     1043.103696
10     727.398895
Name: x, dtype: float64, 0      500.040333
1     1147.365567
2      384.987222
3      408.329135
4      917.990542
5      777.055729
6      607.856477
7      251.874422
8      765.025073
10     732.276032
Name: y, dtype: float64)


ValueError: 'positions' must be a (x, y) pixel position or a list or array of (x, y) pixel positions, e.g., [(x1, y1), (x2, y2), (x3, y3)]

<Figure size 1440x1280 with 0 Axes>

: 

## 8 Perform appeture photometry and add to the results
instrumental_mag is calculated as -2.5 * LOG10(`aperture_sum`)


In [14]:
aperture_radius =6.0
positions = (results['x'], results['y'])    
apertures = CircularAperture(positions, r=aperture_radius)    
phot_table = aperture_photometry(data, apertures)     
results['aperture_sum'] = phot_table['aperture_sum']
# add a col with calculation for instrumental mag
results['instrumental_mag'] = results.apply(lambda x: -2.5 * math.log10(x['aperture_sum']), axis = 1)
results

ValueError: 'positions' must not contain any non-finite (e.g., NaN or inf) positions

## 9 Compute the ensemble using the comparison stars.

**Use nunmpy linear regression on the comparison stars' vmag and instrumental_mag**

We should get a good linear fit with low residuals if the image is of good quality
Once we have the fit, feed the target instrumental_mag into the fit function to determine the magnitude.
Plot the results to visualise the data quality

In [ ]:
# now perform ensemble photometry by linear regression of the comparison stars' instrumental mags
to_linear_fit = results.query('auid != "target" and vmag > {} and vmag < {}'.format(BRIGHTEST_COMPARISON_STAR_MAG, DIMMEST_COMPARISON_STAR_MAG)) 
x = to_linear_fit['instrumental_mag'].values
y = to_linear_fit['vmag'].values
fit, residuals, rank, singular_values, rcond = np.polyfit(x, y, 1, full=True)
fit_fn = np.poly1d(fit) 

## 10 Use the ensemble fit from above to calculate target mag

Plot again for a sanity check

In [ ]:

# fit_fn from above is a function which takes in x and returns an estimate for y, lets feed in the 'target' instrumental mag
target_instrumental_magnitude = results[results.auid=='target']['instrumental_mag'].values[0]
target_magnitude = fit_fn(target_instrumental_magnitude)
print('Magnitude estimate: {} error from residuals {}'.format(target_magnitude, residuals))
x = np.append(x,target_instrumental_magnitude)
y = np.append(y,target_magnitude)

## 11 Use the ensemble to calculate a check star mag

Calculate the mag for a check star, lets pick 000-BML-045 from the plot above

In [ ]:

check_star_instrumental_magnitude = results[results.auid=='000-BML-045']['instrumental_mag'].values[0]
check_magnitude = fit_fn(check_star_instrumental_magnitude)
print('Check star 000-BML-045 magnitude = {}'.format(check_magnitude))

Check star 000-BML-045 magnitude = 12.434656369385017


## 12 Summary for submission to AAVSO

In [ ]:
observation_date = datetime.strptime(header['DATE-OBS'], '%Y-%m-%dT%H:%M:%S.%f')
print('Star Identifier: {}'.format(STAR_NAME))
print('Date-time : {}'.format(observation_date.strftime('%Y/%m/%d/%H/%M/%S')))
print('Magnitude: {} Error: {}'.format(target_magnitude, residuals))
print('Check Star 000-BML-045 Magnitude: {}'.format(check_magnitude))
print('Chart ID: {}'.format(chart_id))
print('Ensemble of {}. Error as residuals of linear fit'.format(to_linear_fit['auid'].values))

Star Identifier: KIC08462852
Date-time : 2017/10/02/20/56/22
Magnitude: 11.894127782624818 Error: [ 0.0013096]
Check Star 000-BML-045 Magnitude: 12.434656369385017
Chart ID: X21328CWX
Ensemble of ['000-BLS-551' '000-BML-045' '000-BLS-555' '000-BML-046']. Error as residuals of linear fit
